In [4]:
import matplotlib.pylab as plt
plt.rcParams["figure.figsize"] = (10, 9)
import scipy.optimize
import pandas as pd
import numpy as np
from numpy.linalg import inv
import numpy.matlib
import math
from IPython.display import Image

In [1]:
import requests
from io import BytesIO

In [5]:
def krx(tdate):
    global health
    gen_req_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    query_str_parms = {
        'idxIndMidclssCd' : '02',
        'trdDd': str(tdate),
        'share' : '2',
        'money' : '3',
        'csvxls_isNo' : 'false',
        'name' : 'fileDown',
        'url' : 'dbms/MDC/STAT/standard/MDCSTAT00101'
    }
    headers = {
        'Referer' :'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101',
        'Upgrade-Insecure-Requests' : '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'
    }
    r = requests.get(gen_req_url, query_str_parms, headers = headers)
    code = r.content
    gen_req_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    form_data = {
        'code' : code
    }
    r = requests.post(gen_req_url, form_data)
    df = pd.read_csv(BytesIO(r.content), encoding = "CP949")
    df['일자'] = tdate
    return df

In [ ]:
for year in range(2019, 2020):
    for month in range(1, 13):
        m = m.iloc[0:0]
        for day in range(1, 32):
            tdate = int(year*10000+month*100+day*1)
            if tdate <= 20191231:
                data = krx(tdate)
                data = data.iloc[7:18, :]
                data = data[['상장시가총액']]
                data = data.transpose()
                m = pd.concat([m, data])

In [11]:
def kospi_construct(start, end):
    m = pd.DataFrame()
    for year in range(start, end+1):
        for month in range(1, 13):
            m = m.iloc[0:0]
            for day in range(1, 32):
                tdate = int(year*10000+month*100+day*1)
                data = krx(tdate)
                data = data.set_index('지수명')
                data = data.loc['코스피 200 건설'][:]
                m = pd.concat([m, data])
            m = m.dropna(axis = 0)
            m_mean = m.mean()
            m_mean = m_mean/m_mean.sum()
            m_mean = m_mean.to_frame()
            m_mean = m_mean.transpose()
            index = str(year)+'/'+str(month)
            m_mean.index = [index]
            total = pd.concat([total, m_mean])

,지수명,종가,대비,등락률,시가,고가,저가,거래량,거래대금,상장시가총액,일자
0,코스피 (외국주포함),NaN,NaN,NaN,NaN,NaN,NaN,310882.0,3200601.0,1.313649e+09,20161223
1,코스피,2035.90,0.17,0.01,2036.17,2037.71,2031.54,309277.0,3197109.0,1.313135e+09,20161223
2,코스피 200,261.61,-0.36,-0.14,262.10,262.26,261.30,46536.0,2263080.0,1.114645e+09,20161223
3,코스피 100,1972.93,-4.12,-0.21,1978.03,1979.18,1970.84,30075.0,1918138.0,1.020917e+09,20161223
4,코스피 50,1720.50,-4.61,-0.27,1726.05,1727.14,1718.48,17621.0,1539773.0,8.742304e+08,20161223
5,코스피 200 중소형주,1007.11,7.36,0.74,999.76,1007.49,996.82,16461.0,344941.0,9.372786e+07,20161223
6,코스피 200 건설,264.15,-0.20,-0.08,264.90,264.90,262.72,2866.0,56669.0,2.520492e+07,20161223
7,코스피 200 중공업,318.81,0.33,0.10,317.57,319.55,315.65,4463.0,88067.0,2.795097e+07,20161223
8,코스피 200 철강/소재,1000.90,-1.59,-0.16,1007.65,1009.89,994.02,1298.0,120386.0,4.720788e+07,20161223
9,코스피 200 에너지/화학,1217.93,5.85,0.48,1215.31,1220.98,1210.15,3161.0,245933.0,1.048241e+08,20161223
